In [ ]:
%pip install -e 'src/hear21passt'

Obtaining file:///root/data/exploratory_notebooks/da/src/hear21passt
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.0/377.0 kB 20.5 MB/s eta 0:00:00
  Running setup.py develop for hear21passt
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install tensorboard

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 61.3 MB/s eta 0:00:00:00:010:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.5/126.5 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 91.1 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.8/181.8 kB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 22.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 76.1 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.3/181.3 kB 29.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 151.7/151.7 kB 35.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import os
os._exit(00)

In [2]:
import sys
import os
import datetime
import re

import numpy as np
import pandas as pd
import torch
import torchaudio
import tqdm

from operator import itemgetter
from hashlib import sha1

from torch.optim import AdamW, lr_scheduler
from torch.utils.data import DataLoader, default_collate
from torch.nn import functional as F
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import ChainDataset

from sklearn.metrics import f1_score

from IPython.display import Audio

from optim.lr_scheduler import get_scheduler_lambda
from utils.model import get_pretrained_passt_model
from utils.gdsc22_dataset import BuzzIterableDataset, BuzzAugmentedIterableDataset
from utils.dataset_augmentations import BuzzIterableTransformedDataset, MixUp1Level, MixUp2Level, RandomGain, Rolling, ColoredNoise

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


## 1. Initial configuration

In [3]:
torch.manual_seed(42)
np.random.seed(42)

In [4]:
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../../'))
data_processed_dir = os.path.join(project_dir, 'gdsc_data/data_processed/')
data_source_dir = os.path.join(project_dir, 'gdsc_data/data_source/')

In [5]:
ts = datetime.datetime.now().strftime('%Y-%m-%dT%H:%M:%S')

MODEL_NAME = f"gdsc22_passt_kd_NOEXP_lr_mismatch_size_t40_f4_epochs40_gain_roll_mixup_noise_batch_mixup@{ts}.pt"
TORCH_SAVE_PATH = os.path.join(project_dir, f'checkpoints/{MODEL_NAME}')

TENSORBOARD_RUNS_PATH = os.path.join(project_dir, "torch_runlogs/")
TENSORBOARD_EXP_FOLDER = f"PaSST_kd_NOEXP_lr_mismatch_size_t40_f4_epochs40_gain_roll_mixup_noise_batch_mixup@{ts}"
TENSORBOARD_EXP_COMMENT = "PaSST_s_kd_p16_128_ap486_NOEXP_lr_gain_roll_mixup_noise_batch_mixup"

TRAIN_BATCH_SIZE = 32
VAL_BATCH_SIZE = 32

# fine tuning and ohe of real labels
CLASSIFIER_N_CLASSES = 66

DATA_TAG = "data_8c86715"

DEBUG_TRANSFORMATIONS = False

In [6]:
#final_dataset_path = find_final_dataset_path(data_processed_dir, DATA_TAG)
final_dataset_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '01_apply_ir_function')
final_dataset_val_as_train_path_augmented = os.path.join(data_processed_dir, DATA_TAG, '03_apply_ir_function_on_validation')

final_dataset_path = data_source_dir
final_dataset_val_as_train_path = os.path.join(data_processed_dir, DATA_TAG, '02_resample_validation/')

In [7]:
train_dir_augmented = os.path.join(final_dataset_path_augmented, 'train/')
val_as_train_dir_augmented = os.path.join(final_dataset_val_as_train_path_augmented, 'train/')

train_dir = os.path.join(final_dataset_path, 'train/')
val_as_train_dir = os.path.join(final_dataset_val_as_train_path, 'train/')

val_dir = os.path.join(final_dataset_val_as_train_path, 'val/')

## 2. Dataloaders and batching

### 2.1 Debugging collate_fn

Used to check the random seed and batchability of the individual points 0:)

In [8]:
def debug_collate(batch):
    get_1st = itemgetter(0)
    files = ", ".join(map(get_1st, batch)).encode('utf-8')
    worker_info = torch.utils.data.get_worker_info()   
    print(f"Worker_id '{worker_info.id}' batch hash is '{sha1(files).hexdigest()}'") 
    
    batch = default_collate(batch)
    return batch

In [9]:
train_dataset = BuzzIterableDataset(
    train_dir,
    metadata_csv=os.path.join(train_dir, 'metadata.csv'), 
    shuffle=True,
    subsample=1.0,
    total_classes=CLASSIFIER_N_CLASSES
)
val_as_train_dataset = BuzzIterableDataset(
    val_as_train_dir,
    metadata_csv=os.path.join(val_as_train_dir, 'metadata.csv'), 
    shuffle=True,
    subsample=1.0,
    total_classes=CLASSIFIER_N_CLASSES
)

train_dataset_transformed = BuzzIterableTransformedDataset(
    transformers=[RandomGain(debug=DEBUG_TRANSFORMATIONS), Rolling(debug=DEBUG_TRANSFORMATIONS), MixUp1Level(debug=DEBUG_TRANSFORMATIONS), ColoredNoise(debug=DEBUG_TRANSFORMATIONS)],
    buzz_iterable=train_dataset
)

val_as_train_dataset_transformed = BuzzIterableTransformedDataset(
    transformers=[RandomGain(debug=DEBUG_TRANSFORMATIONS), Rolling(debug=DEBUG_TRANSFORMATIONS), MixUp1Level(debug=DEBUG_TRANSFORMATIONS), ColoredNoise(debug=DEBUG_TRANSFORMATIONS)],
    buzz_iterable=val_as_train_dataset    
)

# I had too much hope on the IR augmented dataset and took a wrong path...
# train_dataset_augmented = BuzzAugmentedIterableDataset(
#     train_dir_augmented, 
#     metadata_csv=os.path.join(final_dataset_path_augmented, 'metadata_grouped_train.csv'), 
#     shuffle=True,
#     subsample=0.2,
#     total_classes=CLASSIFIER_N_CLASSES
# )
# val_as_train_dataset_augmented = BuzzAugmentedIterableDataset(
#     val_as_train_dir_augmented, 
#     metadata_csv=os.path.join(final_dataset_val_as_train_path_augmented, 'metadata_grouped_train.csv'), 
#     shuffle=True,
#     subsample=0.2,
#     total_classes=CLASSIFIER_N_CLASSES
# )
    
train_dataloader = DataLoader(
    ChainDataset([train_dataset_transformed, val_as_train_dataset_transformed]), 
    batch_size=TRAIN_BATCH_SIZE, 
    pin_memory=True,
    num_workers=2,
    #collate_fn=debug_collate
)

In [10]:
val_dataset = BuzzIterableDataset(
    val_dir, 
    metadata_csv=os.path.join(val_dir, 'metadata.csv'),
    shuffle=False,
    total_classes=CLASSIFIER_N_CLASSES,
)
val_dataloader = DataLoader(val_dataset, batch_size=VAL_BATCH_SIZE)

In [20]:
transformed = next(iter(train_dataloader))[2]

In [41]:
transformed[0]

tensor([-0.0061, -0.0143, -0.0044,  ...,  0.0244, -0.0471,  0.0270])

In [48]:
Audio(transformed[14], rate=32000)

## 3. Init pytorch objects

In [10]:
writer = SummaryWriter(os.path.join(TENSORBOARD_RUNS_PATH, TENSORBOARD_EXP_FOLDER), comment=TENSORBOARD_EXP_COMMENT)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [11]:
%%sh
nvidia-smi

sh: 1: nvidia-smi: not found


CalledProcessError: Command 'b'nvidia-smi\n'' returned non-zero exit status 127.

In [12]:
# Default stride for t and f is 10
# NOTE: FMAX is used in frequency augmentation. Every sample has a random subset of fmin and fmax applied to it...
model = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=40, s_patchout_f=4)

#model = get_pretrained_passt_model(arch='passt_s_swa_p16_128_ap476', mode="all", n_classes=66, s_patchout_t=10, s_patchout_f=3)
model.to(device)



 Loading PaSST pre-trained on AudioSet (with KD) Patch 16 stride 10 structured patchout mAP=486 


PaSST(
  (patch_embed): PatchEmbed(
    (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
    (norm): Identity()
  )
  (pos_drop): Dropout(p=0.0, inplace=False)
  (blocks): Sequential(
    (0): Block(
      (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (attn): Attention(
        (qkv): Linear(in_features=768, out_features=2304, bias=True)
        (attn_drop): Dropout(p=0.0, inplace=False)
        (proj): Linear(in_features=768, out_features=768, bias=True)
        (proj_drop): Dropout(p=0.0, inplace=False)
      )
      (drop_path): Identity()
      (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
      (mlp): Mlp(
        (fc1): Linear(in_features=768, out_features=3072, bias=True)
        (act): GELU(approximate='none')
        (fc2): Linear(in_features=3072, out_features=768, bias=True)
        (drop): Dropout(p=0.0, inplace=False)
   

PaSSTGDSCIface(
  (mel): AugmentMelSTFT(
    winsize=800, hopsize=320
    (freqm): FrequencyMasking()
    (timem): TimeMasking()
  )
  (net): PaSST(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(1, 768, kernel_size=(16, 16), stride=(10, 10))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=768, out_features=768, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (drop_path): Identity()
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=768, out_features=3072, bias=True)
          (act): GELU(approximate='none')
          (fc2): Linear(in_features=307

In [15]:
Mixup2LevelScaler = MixUp2Level(debug=True)

for _, _, X_batch, y in train_dataloader:
    X_batch_spectr = model.mel(X_batch).unsqueeze(1)
    
    handle = Mixup2LevelScaler(X_batch_spectr, y, device)
    X_batch_spectr = handle.send(None)
    
    try:
        loss, y_hat = handle.send((F.cross_entropy, y))
    except StopIteration:
        pass
    
    break
    

<class 'utils.dataset_augmentations.MixUp2Level'>: 2LvlMixup (1st part) lambda=tensor([0.9959, 0.9997, 0.7584, 0.6537, 0.9976, 0.9924, 1.0000, 1.0000, 0.9950,
        0.9737, 0.7179, 1.0000, 0.9399, 0.7808, 0.7165, 0.9992, 0.8143, 0.9236,
        0.9991, 0.5090, 0.9271, 0.9164, 0.9879, 0.9247, 0.9823, 0.9952, 0.9048,
        0.9945, 0.9886, 0.9987, 0.9756, 0.9841])
<class 'utils.dataset_augmentations.MixUp2Level'>: 2LvlMixup (2nd part) y_mix=tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
<class 'utils.dataset_augmentations.MixUp2Level'>: 2LvlMixup samples_loss=tensor([3.3746, 3.2157, 3.4569, 3.5616, 3.6393, 3.5781, 3.3250, 3.2154, 3.3670,
        3.2335, 3.5539, 3.2154, 3.2754, 3.3199, 3.2154, 3.2162, 3.4011, 3.3626,
        3.4059, 3.2154, 3.4587, 3.2990, 3.2154, 3.2232, 3.2154, 3.

In [ ]:
adamw = AdamW(model.net.parameters(), lr=0.00001, weight_decay=0.0001)
lr_scheduler = torch.optim.lr_scheduler.LambdaLR(adamw, get_scheduler_lambda(warm_up_len=0))

scaler = torch.cuda.amp.GradScaler()


Mixup2LevelScaler = MixUp2Level()

print(f"Initial Learning rate:{lr_scheduler.get_last_lr()[0]}")
for epoch in range(0, 40):
    y_tr_pred_f1 = []
    y_tr_true_f1 = []
    train_loss = []

    model.train()
    for _, _, X_batch, y in (progress := tqdm.tqdm(train_dataloader)):
        adamw.zero_grad()        
        
        X_batch = X_batch.to(device)
        y = y.to(device)
        X_batch_spectr = model.mel(X_batch).unsqueeze(1)
               
        with  torch.cuda.amp.autocast():
            # Start processing with coroutine
            handle = Mixup2LevelScaler(X_batch_spectr, y, device)
            X_batch_spectr = handle.send(None)
            y_hat, emb = model.net(X_batch_spectr)
            try:
                loss, y_hat = handle.send(F.cross_entropy, y_hat)
            except StopIteration:
                pass
            
            loss = loss.mean()            
            train_loss.append(loss.detach().cpu())
            progress.set_description(f"TrainL {float(loss):.2f}")
            
        # Scales loss.  Calls backward() on scaled loss to create scaled gradients.
        # Backward passes under autocast are not recommended.
        # Backward ops run in the same dtype autocast chose for corresponding forward ops.
        scaler.scale(loss).backward()
        # scaler.step() first unscales the gradients of the optimizer's assigned params.
        # If these gradients do not contain infs or NaNs, optimizer.step() is then called,
        # otherwise, optimizer.step() is skipped.
        scaler.step(adamw)
        # Updates the scale for next iteration.
        scaler.update()
        
        prob = F.softmax(y_hat.detach(), dim=1)
        y_pred = torch.argmax(prob.detach(), dim=1)
        y_tr_pred_f1.append(y_pred.cpu())
        y_tr_true_f1.append(y.cpu().argmax(dim=1))

    f_score_train = f1_score(torch.cat(y_tr_true_f1).numpy(), torch.cat(y_tr_pred_f1).numpy(), average='macro')
    total_train_loss = torch.stack(train_loss, dim=0).mean()
    print(f"=" * 80)
    print("\r\n")
    print(f"Train Loss(epoch={epoch}): {total_train_loss}")
    print(f"F1 score(train) {f_score_train}")
    print("\r\n")
    print(f"=" * 80)
    print("\r\n")

    val_loss = []
    y_val_pred_f1 = []
    y_val_true_f1 = []
    model.eval()
    
    with torch.no_grad():
        for _, _, X_batch, y_val in (progress := tqdm.tqdm(val_dataloader)):
            X_batch = X_batch.to(device)
            y_val = y_val.to(device)
            X_batch_spectr = model.mel(X_batch).unsqueeze(1)

            with torch.cuda.amp.autocast():
                y_hat, emb = model.net(X_batch_spectr)
            
                _loss = F.cross_entropy(y_hat, y_val, reduction="none")
                _loss = _loss.mean()
                val_loss.append(_loss.detach().cpu())
            
            progress.set_description(f"ValL {float(_loss):.2f}")            

            prob = F.softmax(y_hat.detach(), dim=1)
            y_pred = torch.argmax(prob.detach(), dim=1)

            y_val_pred_f1.append(y_pred.cpu())
            y_val_true_f1.append(y_val.cpu().argmax(dim=1))
            
    f1_score_val = f1_score(torch.cat(y_val_true_f1).numpy(), torch.cat(y_val_pred_f1).numpy(), average='macro')
    total_val_loss = torch.stack(val_loss, dim=0).mean()
    print(f"=" * 80)
    print("\r\n")
    print(f"Val Loss: {total_val_loss}")
    print(f"F1 score(val) {f1_score_val}")
    print("\r\n")    
    print(f"=" * 80)
    print("\r\n")
       
    writer.add_scalar('Loss/train', total_train_loss, epoch)
    writer.add_scalar('Loss/val', total_val_loss, epoch)
    writer.add_scalar('F1/train', f_score_train, epoch)
    writer.add_scalar('F1/val', f1_score_val, epoch)
    writer.add_scalar('LR', lr_scheduler.get_last_lr()[0], epoch)

    lr_scheduler.step()
    print(f"New lr: {lr_scheduler.get_last_lr()[0]}")
       
    torch.save(
        {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': adamw.state_dict(),
        }, TORCH_SAVE_PATH)

Initial Learning rate:1e-05


0it [00:00, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


x torch.Size([32, 1, 128, 1000])
self.norm(x) torch.Size([32, 768, 12, 99])
 patch_embed :  torch.Size([32, 768, 12, 99])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 99])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 12, 1])
X Before time Patchout of 40  torch.Size([32, 768, 12, 99])
X after time Patchout torch.Size([32, 768, 12, 59])
X Before Freq Patchout of 4  torch.Size([32, 768, 12, 59])
 
 X after freq Patchout:  torch.Size([32, 768, 8, 59])
X flattened torch.Size([32, 472, 768])
 self.new_pos_embed.shape torch.Size([1, 2, 768])
 self.cls_tokens.shape torch.Size([32, 1, 768])
 self.dist_token.shape torch.Size([32, 1, 768])
 final sequence x torch.Size([32, 474, 768])
 after 12 atten blocks x torch.Size([32, 474, 768])
forward_features torch.Size([32, 768])
head torch.Size([32, 66])


TrainL 4.06: : 450it [06:31,  1.15it/s]




Train Loss(epoch=0): 3.9156384468078613
F1 score(train) 0.043291614639326195






...

0it [00:00, ?it/s]/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
TrainL 0.71: : 450it [06:36,  1.14it/s]




Train Loss(epoch=18): 0.8780385851860046
F1 score(train) 0.7711581204422031






0it [00:00, ?it/s]/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
ValL 0.56: : 16it [00:18,  1.17s/it]




Val Loss: 0.35561034083366394
F1 score(val) 0.7881184591092909




New lr: 1e-05


0it [00:00, ?it/s]/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")
TrainL 0.98: : 59it [00:52,  1.18it/s]

In [ ]:
model_eval = get_pretrained_passt_model(mode="all", n_classes=66, s_patchout_t=0, s_patchout_f=0)
state = torch.load(TORCH_SAVE_PATH)
model_eval.load_state_dict(state['model_state_dict'])

In [7]:
model_eval = model_eval.cuda()

In [8]:
    val_loss = 0.0
    ypred_f1 = []
    ytrue_f1 = []
    model_eval.eval()
    
    with torch.no_grad():
        for batch, y_val in tqdm.tqdm(val_dataloader):
            batch = batch.cuda()
            y_val = y_val.cuda()

            y_hat, emb = model_eval(batch)
            _loss = F.cross_entropy(y_hat, y_val, reduction="none")

            prob = F.softmax(y_hat, dim=1)
            y_pred = torch.argmax(prob, dim=1)

            ypred_f1.append(y_pred.cpu())
            ytrue_f1.append(y_val.cpu())

            val_loss += _loss.sum()

    f1_score_val = f1_score(torch.cat(ytrue_f1).numpy(), torch.cat(ypred_f1).numpy(), average='macro')

    print(f"Val Loss: {val_loss}")
    print(f"=" * 80)
    print(f"F1 score(val) {f1_score_val}")
    print(f"=" * 80)

  0%|          | 0/169 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/torch/functional.py:632: UserWarning: stft will soon require the return_complex parameter be given for real inputs, and will further require that return_complex=True in a future PyTorch release. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:801.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
/root/data/exploratory_notebooks/da/src/hear21passt/hear21passt/models/passt.py:292: UserWarning: Input image size (128*1000) doesn't match model (128*998).
  warnings.warn(f"Input image size ({H}*{W}) doesn't match model ({self.img_size[0]}*{self.img_size[1]}).")


x torch.Size([12, 1, 128, 1000])
self.norm(x) torch.Size([12, 768, 12, 99])
 patch_embed :  torch.Size([12, 768, 12, 99])
 self.time_new_pos_embed.shape torch.Size([1, 768, 1, 99])
 self.freq_new_pos_embed.shape torch.Size([1, 768, 12, 1])
X flattened torch.Size([12, 1188, 768])
 self.new_pos_embed.shape torch.Size([1, 2, 768])
 self.cls_tokens.shape torch.Size([12, 1, 768])
 self.dist_token.shape torch.Size([12, 1, 768])
 final sequence x torch.Size([12, 1190, 768])
 after 12 atten blocks x torch.Size([12, 1190, 768])
forward_features torch.Size([12, 768])
head torch.Size([12, 66])


100%|██████████| 169/169 [03:09<00:00,  1.12s/it]

Val Loss: 1069.6846923828125
F1 score(val) 0.8392231749961314


In [10]:
%%sh
nvidia-smi

Thu Jul  6 18:38:30 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:1E.0 Off |                    0 |
| N/A   43C    P0    28W /  70W |  15010MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------


## Miscellaneous

### Glossary

SWA - Stochastic weight averaging

### Model stored weights
```
Downloading: "https://github.com/kkoutini/PaSST/releases/download/v0.0.1-audioset/passt-s-f128-p16-s10-ap.476-swa.pt" to /root/.cache/torch/hub/checkpoints/passt-s-f128-p16-s10-ap.476-swa.pt
```